#User based collaborative filtering

- Use The Last.FM dataset
- Use cosine similarity


In [1]:
#import libraries
import pandas as pd
from scipy.spatial.distance import cosine

In [2]:
#Read csv file
data = pd.read_csv('data/lastfm-matrix-germany.csv')

#check dataset with specific slice of data
data.head(6).iloc[:,2:8]

,abba,ac/dc,adam green,aerosmith,afi,air
0,0,0,0,0,0,0
1,0,0,1,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0


In [3]:
#drop col named 'user'
data_germany = data.drop('user', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [4]:
#Create dataframe list (items vs items)
data_ibs = pd.DataFrame(index= data_germany.columns, columns = data_germany.columns)
data_ibs

,a perfect circle,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
a perfect circle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ac/dc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
adam green,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aerosmith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trivium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
underoath,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
volbeat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#each item neighbour by looping through data_ibs in descending order
data_neighbours = pd.DataFrame(index= data_ibs.columns, columns = range(1, 11))
data_neighbours

,1,2,3,4,5,6,7,8,9,10
a perfect circle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ac/dc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
adam green,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aerosmith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
trivium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
underoath,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
volbeat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#fun to get similarity scores, sum of product 2 vectors containing purchase history and item similarities and divid by the sum of the similarities in the respective vector
def getScore(history, similarities):
  return sum(history*similarities)/ sum(similarities)

#data_sims and data are basically same but with nothing filled in except the header
data_sims = pd.DataFrame(index = data.index, columns = data.columns)
data_sims.iloc[:,:1] = data.iloc[:,:1]


In [8]:
#loop rows and columns and fill in empty space with similarity scores
for i in range(0, len(data_sims.index)):
  for j in range(1, len(data_sims.columns)):
    user = data_sims.index[i]
    product = data_sims.columns[j]
    
    if data.iloc[i, j] == 1:
      data_sims.iloc[i, j] = 0
    else:
      product_top_names = data_neighbours.loc[product][1:10]
      product_top_sims = data_ibs.loc[product].sort_values(ascending = False)[1:10]
      #user_purchases = data_germany.loc[user,product_top_names]
      user_purchases = data_germany.loc[user]
      

      data_sims.iloc[i, j] = getScore(user_purchases, product_top_sims)


In [9]:
#Get the top songs
data_recommend = pd.DataFrame(index = data_sims.index, columns = ['user','1','2','3','4','5','6'])
data_recommend.iloc[0:,0] = data_sims.iloc[:,0]


In [10]:
#instead of top song scores, to see names
for i in range(0, len(data_sims.index)):
  data_recommend.iloc[i,1:] = data_sims.iloc[i,:].sort_values(ascending = False).iloc[1:7,].index.transpose()
  

#Print sample
print(data_recommend.iloc[:10,:4])  

  user                 1                 2              3
0    1  dropkick murphys         eluveitie      goldfrapp
1   33             keane       the strokes      the kooks
2   42    as i lay dying     avril lavigne    evanescence
3   51    arctic monkeys    the ting tings  the fratellis
4   62        beatsteaks      billy talent      blink-182
5   75      alexisonfire               atb   billy talent
6  130         tori amos  a perfect circle           abba
7  141  all that remains       tenacious d     stone sour
8  144            beirut         cocorosie       interpol
9  150    arctic monkeys         deichkind      rammstein
